# guide_bot

Guide design is a difficult task as we don't have analytical tools that can predict what shape would be best. Currently we rely on numerical optimizations of ray-tracing simulations such as McStas.

For each new idea we need a significant effort to write a simulation, connect it to a numerical optimizer and investigate the results.

guide_bot is a piece of software that attempts to simplify this greatly.

Was written by Mads Bertelsen as a master project at University of Copenhagen. It is now being rewritten by Mads Bertelsen as part of the HighNESS EU project.

![alt](figures/powerpoint_figures/Slide01.png)

## Guides use super mirrors to transport neutrons
![alt](figures/powerpoint_figures/Slide02.png)

## MATLAB guide_bot
- Developed 2013-2015
- Allows guide design without knowing any McStas
- MATLAB code (requires license)
- Difficult to install
- Used to select moderator at ESS
- Designed guides for BIFROST, HEIMDAL, PSI CAMEA, PSI FOCUS
- Used at NIST in the USA
- Was expanded to handle coating optimization by Martin Olsen

![alt](figures/powerpoint_figures/Slide05.png)

## Python guide_bot
- Currently being developed
- Uses McStasScript as dependency
- More flexible and scalable
- Much easier to install

### Install newest version on e-learning platform
Open the terminal and type:
```
pip install guide_bot --upgrade --user
```

# Demo of guide_bot

In [ ]:
import guide_bot as gb

### Describe target for beam
Often describing the sample at a time of flight instrument, or monochromator at continuous source. 

![alt](figures/powerpoint_figures/Slide03.png)

### Describe target for beam
Inserting this into guide_bot takes the following form. Adding instrument length and guide - target distance.

In [ ]:
target = gb.Target(width=0.01, height=0.02,
                   div_horizontal=0.75, div_vertical=1.2,
                   min_wavelength=1.5, max_wavelength=3.0,
                   instrument_length=60, target_guide_distance=0.5)

### Optimizing for brilliance transfer
![alt](figures/powerpoint_figures/Slide04.png)

### Describe the neutron source
![alt](figures/powerpoint_figures/Slide06.png)

### Description in guide_bot

In [ ]:
moderator = gb.Moderator(name="fom_moderator", width=0.1, height=0.05, guide_start=2.0)

### Create a guide_bot project
- Settings:
    - settings for optimizer
    - settings for McStas simulations
    
- Collects target and moderator
- Provides name which will be used as folder name!

In [ ]:
settings = {"maxiter": 5, "swarmsize": 3, "minstep": 2E-4,
            "ncount": 1E6, "ncount_analysis": 3E6, "logfile": True}

project = gb.Project(name="simple_project", target=target, moderator=moderator, settings=settings)

### Settings
| Element          | Explanation                                                     |
| :---             |    :----                                                        |
| maxiter          | Maximum number of iterations (of each swarm particle)           |
| swarmsize        | Number of swarm particles                                       |
| minstep          | Minimum allowed step (probably in normalized parameters)        |
| ncount           | Number of rays for optimization                                 |
| ncount_analysis  | Number of rays for analysis                                     |
| logfile          | Whether to write a log file, always keep this to True!          |

Uses pyswarm for optimization [https://pythonhosted.org/pyswarm/](https://pythonhosted.org/pyswarm/), all options are available through settings, only the most important ones mentioned above.

### Add a guide
One can add as many guides to a project as desired. Request a guide object from project and build on it.

In [ ]:
guide = project.new_guide(name="simplest_guide")
guide += gb.Straight(name="long_guide", m=4)

### Look at guide object

In [ ]:
print(guide)

## Available elements
New elements are easy to add, more are in development!

| Element    | Explanation                                                     |
| :---       |    :----                                                        |
| Straight   | Straight mirrors, can have different start and end dimensions   |
| Elliptic   | Elliptic guide element                                          |
| Gap        | Gap in the guide (no mirrors or cut of at start or end)         |
| Slit       | Gap in the guide (no mirrors, cut off at start)                 |
| Kink       | Gap with change of direction at end                             |
| Curved     | Curved guide, same dimensions at start and end (no gravity)     |

### Element options
All elements in guide_bot have the following options.

| Parameter    | Unit  | Explanation                                                     |
| :---         | :---  |    :----                                                        |
| length       | [m]   | Length of the guide element                                     |
| start_point  | [m]   | Distance from source to start of element (measured along guide) |
| start_width  | [m]   | Width of element at the start                                   |
| start_height | [m]   | Height of element at the start                                  |
| end_width    | [m]   | Width of the element at the end                                 |
| end_height   | [m]   | Height of the element at the end                                |

Each of them can be:
- Left unset (guide_bot will select reasonable range for optimization)
- Have a fixed value given
- Have a range given (None for either end makes guide_bot choose that end)

### Add another guide

In [ ]:
guide = project.new_guide(name="Elliptic_guide")
guide += gb.Elliptic(name="Transport", m=4, start_width=[0.01, 0.04])

### Multiple elements in a guide
![alt](figures/guide_elements.png)

### And yet another guide

In [ ]:
guide = project.new_guide(name="Combined_guide")
guide += gb.Elliptic(name="Extraction", m=4)
guide += gb.Gap(name="Chopper_gap", start_point=[6.3, 6.8], length=0.1)
guide += gb.Elliptic(name="Transport", m=4)

### View project
These guides are all remembered by the project object.

In [ ]:
print(project)

### Write project
The project needs to be written to disk in order to execute simulations.


In [ ]:
project.write()

This creates a folder on in the current work directory with all the code to optimize the described guides for the selected source and beam target parameters!

### A look at the project folder

In [ ]:
ls

## Run the simulation
The simulation tasks are stored as 'pickled' python objects that are executed with gb.RunFromFile.

In [ ]:
import os
cpath = os.getcwd()
os.chdir(os.path.join(cpath, "simple_project", "Elliptic_guide"))
gb.RunFromFile("Elliptic_guide.plk")

## View the results
After a guide_bot run is complete, you can view the results by making a results object.

In [ ]:
results = gb.Results(project.name) # Provide path to project folder

In [ ]:
%matplotlib widget
results.plot_guide()

In [ ]:
results.plot_any_monitor()

## Conclusion
Simple steps to perform guide optimization.
- Define a target beam
- Define a source
- Create a project
- Add any number of guides
- Write project to disk
- Run project
- Look at results

No need to know McStas, McStasScript. Just basic Python.